In [1]:
pip install Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 47.8 MB/s eta 0:00:00


In [2]:
pip install torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.0 MB/s eta 0:00:00


In [5]:
import pandas as pd


In [6]:
pip install scikit-learn

In [7]:
df = pd.read_csv('/content/drive/MyDrive/dataset.csv', encoding='ISO-8859-1')
df.shape

half_size = len(df) // 5
trimmed_df = df.head(half_size)
trimmed_df.shape
df=trimmed_df

In [8]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True)
train_df.shape, test_df.shape

((180000, 2), (20000, 2))

In [9]:
half_size = len(test_df) // 20
trimmed_df = test_df.head(half_size)
trimmed_df.shape
df=trimmed_df
df.shape

(1000, 2)

In [10]:
pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.8 MB/s eta 0:00:00


In [16]:
from transformers.tools.agents import AutoTokenizer
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer
model_name = '/content/drive/MyDrive/t5-small-finetuned-full/t5-small.zip'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)


In [17]:
df.describe

<bound method NDFrame.describe of                                                     input  \
92841      'J 351 malaria,I filaria & pest control staff.   
56513   This does not include homes that the big banks...   
12905   Double Day Bed Make Through Storage Intended F...   
11002   politicians and government Bureaucrat s are in...   
111956  Responsible for members alone which foreman on...   
...                                                   ...   
154783  The perest hyaluranic acid of nonanimal origin...   
73615   Bees defender Julian Jeanvier had put the host...   
70995   They invented metalworking, copper-working, gl...   
10331   This plant grow in a compact way with a very s...   
168301  In the individual events, Brad Tandy finished ...   

                                                   output  
92841        J 351 malaria, filaria & pest control staff.  
56513   This does not include homes that the big banks...  
12905   Double Day Bed Daybed With Storage Intended F

In [18]:
def correct_grammar(input_text,num_return_sequences=1):
  device = torch.device("cuda" if torch.cuda.is_available() else "cuda")
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text[0]

In [19]:
import pandas as pd

# Create a new DataFrame by applying the function to each row of the original DataFrame 'df'
new_df = df.copy()  # Copy the original DataFrame to a new one

# Define the function to correct grammar
def correct_grammar_for_row(row):
    row['output_predicted'] = correct_grammar(row['input'], num_return_sequences=1)
    return row

# Apply the function to the new DataFrame
new_df = new_df.apply(correct_grammar_for_row, axis=1)

# Print the new DataFrame
print(new_df)


                                                    input  \
92841      'J 351 malaria,I filaria & pest control staff.   
56513   This does not include homes that the big banks...   
12905   Double Day Bed Make Through Storage Intended F...   
11002   politicians and government Bureaucrat s are in...   
111956  Responsible for members alone which foreman on...   
...                                                   ...   
154783  The perest hyaluranic acid of nonanimal origin...   
73615   Bees defender Julian Jeanvier had put the host...   
70995   They invented metalworking, copper-working, gl...   
10331   This plant grow in a compact way with a very s...   
168301  In the individual events, Brad Tandy finished ...   

                                                   output  \
92841        J 351 malaria, filaria & pest control staff.   
56513   This does not include homes that the big banks...   
12905   Double Day Bed Daybed With Storage Intended Fo...   
11002   politicians and

In [21]:
df.to_csv('datafinal-t5small.csv', index=False)  # Set index=False to exclude the index column


In [22]:
import pandas as pd
import Levenshtein



# Function to calculate Levenshtein distance and similarity score
def calculate_similarity(row):
    original_output = row["output"]
    predicted_output = row["output_predicted"]

    # Calculate the Levenshtein distance
    distance = Levenshtein.distance(original_output, predicted_output)

    # Calculate a similarity score (normalized Levenshtein similarity)
    max_len = max(len(original_output), len(predicted_output))
    similarity = 1 - (distance / max_len)

    return similarity

# Apply the calculate_similarity function to each row and store the result in a new column
new_df['similarity_score'] = new_df.apply(calculate_similarity, axis=1)

# Print the DataFrame with the similarity scores
print(new_df)

                                                    input  \
92841      'J 351 malaria,I filaria & pest control staff.   
56513   This does not include homes that the big banks...   
12905   Double Day Bed Make Through Storage Intended F...   
11002   politicians and government Bureaucrat s are in...   
111956  Responsible for members alone which foreman on...   
...                                                   ...   
154783  The perest hyaluranic acid of nonanimal origin...   
73615   Bees defender Julian Jeanvier had put the host...   
70995   They invented metalworking, copper-working, gl...   
10331   This plant grow in a compact way with a very s...   
168301  In the individual events, Brad Tandy finished ...   

                                                   output  \
92841        J 351 malaria, filaria & pest control staff.   
56513   This does not include homes that the big banks...   
12905   Double Day Bed Daybed With Storage Intended Fo...   
11002   politicians and

In [23]:
average_similarity = new_df['similarity_score'].mean()
print(average_similarity)

0.8490550393765096
